In [21]:
import pandas as pd
import sqlite3
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

In [22]:
df = pd.read_excel("50_Startups.xlsx")
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [23]:
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("&", "and")
    .str.replace("-", "_")
)

print("Cleaned Columns:", df.columns.tolist())

Cleaned Columns: ['RandD_Spend', 'Administration', 'Marketing_Spend', 'State', 'Profit']


In [24]:
conn = sqlite3.connect("data.db")

df.to_sql("data_table", conn, if_exists="replace", index=False)

print("Database created successfully ✅")

Database created successfully ✅


In [25]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded ✅")

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


Model loaded ✅


In [28]:
question = "Which state has highest total profit?"

prompt = f"""
You are a SQLite expert.

IMPORTANT RULES:
- Output ONLY raw SQL.
- Do NOT use markdown.
- Do NOT use ``` or ```sql
- Do NOT explain anything.
- Return only one SQL query.

Table name: data_table
Columns: {list(df.columns)}

Question: {question}

SQL:
"""

In [29]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.0,
    do_sample=False
)

sql_query = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
).strip()

print(sql_query)

SELECT State, SUM(RANDD_Spend + Administration + Marketing_Spend) as Total_Profit
FROM data_table
GROUP BY State
ORDER BY Total_Profit DESC
LIMIT 1;


In [30]:
cursor = conn.cursor()
cursor.execute(sql_query)
result = cursor.fetchall()

print("Final Answer:", result)

Final Answer: [('Florida', 7197063.44)]
